In [ ]:
import numpy
import itertools
import torch
from fairseq.data.data_utils import collate_tokens
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

C:\Users\User\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2022-07-28 16:10:59 | INFO | numexpr.utils | NumExpr defaulting to 8 threads.


In [6]:
txt="""Правительство Российской Федерации постановляет:
1. Утвердить прилагаемые изменения, которые вносятся в акты Правительства Российской Федерации.
2. Реализация полномочий, предусмотренных настоящим постановлением, осуществляется в пределах установленной Правительством Российской Федерации численности работников центрального аппарата и территориальных органов исполнительной власти, а также бюджетных ассигнований, предусмотренных соответствующим федеральным органам исполнительной власти в федеральном бюджете на руководство и управление в сфере установленных функций.
"""

In [13]:
tokens = txt.strip( ).split('\n')
for i in range(len(tokens)):
            #limit 200 tokens
    tokens[i]=tokens[i][:200]
numoftoken = len(tokens)
print ('*******',numoftoken)
batch_of_pairs= list(itertools.combinations(tokens,2))
num_batch_of_pairs=len(batch_of_pairs)
print(num_batch_of_pairs)
out=''
if num_batch_of_pairs == 0:
    out = 'no pair to compair'
    print(out)
device = "cuda" if torch.cuda.is_available() else "cpu"
roberta.to(device)
resh=[]
for pair in batch_of_pairs:
    batch =  collate_tokens([roberta.encode(pair[0], pair[1])], pad_idx=1)
    logprobs = roberta.predict('mnli', batch)
    resh.append(logprobs.argmin(dim=1).detach().numpy())

******* 3
3


In [15]:
"""
    0-conflict
    1-no connection
    2-result
"""
fillchar="_"
outp=[]
outp1=numpy.array
for i in range(int((numoftoken*(numoftoken-1))/2)):
    outp.append( '|'+batch_of_pairs[i][0].ljust(50,fillchar)[:50]+'|'+batch_of_pairs[i][1].ljust(50,fillchar)[:50]+'|'+str(resh[i])+'|'+'\n')
    outp1( [batch_of_pairs[i][0].ljust(50,fillchar)[:50],batch_of_pairs[i][1].ljust(50,fillchar)[:50],str(resh[i])])


out = ''.join(outp)
print(out[1:-2])


Правительство Российской Федерации постановляет:__|1. Утвердить прилагаемые изменения, которые вносят|[0]|
|Правительство Российской Федерации постановляет:__|2. Реализация полномочий, предусмотренных настоящи|[0]|
|1. Утвердить прилагаемые изменения, которые вносят|2. Реализация полномочий, предусмотренных настоящи|[2]
